In [23]:
from sklearn import metrics, random_projection, cross_validation, datasets, grid_search, linear_model, metrics, random_projection, manifold, preprocessing
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from collections import  Counter
import numpy as np
import pandas as pd
from sklearn.decomposition import RandomizedPCA
from sklearn import manifold
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.svm import SVC, OneClassSVM
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
from scipy import load, save
import sklearn
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [24]:
dataframe = pd.read_csv('dataframe/genres_40.csv')
dataframe.loc[:, 'm1':] = preprocessing.normalize(dataframe.loc[:, 'm1':])

labels = ['hiphop', 'neoclassic', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
labels_n = ['neoclassic', 'jazz',  'metal', 'pop']

# dataframe = dataframe[dataframe['tag'] == 'neoclassic'].append(dataframe[dataframe['tag'] == 'jazz']).append(dataframe[dataframe['tag'] == 'pop']).append(dataframe[dataframe['tag'] == 'metal'])
# dataframe.append(dataframe[dataframe['tag'] == 'pop'])
# dataframe = dataframe.append(dataframe[dataframe['tag'] == 'jazz'])
# dataframe = dataframe.append(dataframe[dataframe['tag'] == 'metal'])
dataframe = dataframe.sample(frac=1)

shape(dataframe)

(1601, 201)

In [25]:
def search_abnom(dataframe, label, g, nu):
    svm_ = OneClassSVM(gamma=g, nu=nu)
    svm_.fit(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    l = svm_.predict(dataframe[dataframe['tag'] == label].loc[:, 'm1':])
    return l

labels = ['hiphop', 'classical', 'jazz', 'rock', 'metal', 'synth', 'pop', 'blues']
for label in labels:
    svm_label = search_abnom(dataframe, label, 2, 0.15).tolist()
    dataframe[dataframe['tag'] == label] = dataframe[dataframe['tag'] == label][np.array(svm_label) == 1]
    dataframe = dataframe.dropna()
print(shape(dataframe))

(1351, 201)


In [26]:
label_dict = {'hiphop':0, 'classical':1, 'jazz':2, 'rock':3, 'metal':4, 'synth':5, 'pop':6, 'blues': 7} 
int_label = []
for label in dataframe['tag']:
    int_label.append(label_dict[label])

In [27]:
# svm_label = []
# for label in labels:
#     svm_label += search_abnom(label, 2, 0.01).tolist()
# dataframe_svm = dataframe[np.array(svm_label) == 1]

# dataframe_svm_t = np.array(dataframe_svm.loc[:, 'm1':])
# dataframe_svm_l = np.array(dataframe_svm['tag'])

dataframe_svm_t = np.array(dataframe.loc[:, 'm1':])
dataframe_svm_l = np.array(int_label)
dataframe_t = dataframe.loc[:, 'm1':]
np.shape(dataframe_svm_l)

(1351,)

In [28]:
svcc = SVC(C=10, gamma=5, kernel = 'rbf', probability=True)
lrc = LogisticRegression(C = 430)
knc = KNeighborsClassifier(weights = 'distance', n_neighbors=7, metric = 'manhattan')
rfc = RandomForestClassifier(max_depth=23, n_estimators=1300, random_state=1)
xgbc = xgb.XGBClassifier(silent=False, nthread=4, max_depth=2, n_estimators=4000, learning_rate=0.009)

In [20]:
def max_cl(arr):
    list_max_i = []
    for a in arr:
        max_el = 0
        max_i = 0
        for i in range(len(a)):
            if a[i] > max_el:
                max_el = a[i]
                max_i = i
        list_max_i += [max_i]
    return list_max_i


def cross_preds(list_model, list_param, list_save_name):
    list_preds_f = np.array([])
    list_labels_f = np.array([])

    for train_ind, test_ind in cross_validation.StratifiedKFold(dataframe_svm_l, n_folds=5, shuffle=True):
        np.random.shuffle(train_ind) 
        np.random.shuffle(test_ind)
        
        df_ = dataframe_svm_t[:, list_param[0]] 
        list_model[0].fit(df_[train_ind], dataframe_svm_l[train_ind])
        list_preds = list_model[0].predict_proba(df_[test_ind])
        
        for i in range(1, len(list_model)):
            df_ = dataframe_svm_t[:, list_param[i]] 
            list_model[i].fit(df_[train_ind], dataframe_svm_l[train_ind])
            list_preds = np.hstack((list_preds, list_model[i].predict_proba(df_[test_ind])))
            
        if len(list_preds_f) == 0:
            list_preds_f = list_preds
            list_labels_f = dataframe_svm_l[test_ind]
        else:
            list_preds_f = np.vstack((list_preds_f, list_preds))
            list_labels_f = np.hstack((list_labels_f, dataframe_svm_l[test_ind]))

    for i in range(len(list_save_name)):
        save(list_save_name[i], list_preds_f[:, i*8:(i+1)*8])
    save('labels.npy', list_labels_f)                        

In [29]:
metrics.accuracy_score(load('labels.npy'), 
                       max_cl(3*load('svcc.npy') + 3*load('rfc.npy') + 2*load('lrc.npy')))

0.8915929203539823

In [30]:
from scipy.optimize import minimizefc

In [35]:
def res_func(x1, x2, x3):
    return -metrics.accuracy_score(load('labels.npy'), 
                                   max_cl(x1*load('rfc.npy') + x2*load('svcc.npy') + x3*load('lrc.npy')))

def res_cl_report(x1, m1, x2,  m2, x3,  m3):
    print(metrics.classification_report(load('labels.npy'), 
                                   max_cl(x1*m1 + x2*m2)))
    
def s_func(x):
    return res_func(*x)


In [36]:
x0 = (0,0,0)
bnds = ((0, 1), (0, 1), (0, 1))
res = minimize(s_func, x0, method='Nelder-Mead', bounds = bnds)
res

/home/tosha/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:430: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  RuntimeWarning)


 final_simplex: (array([[0.00038735, 0.00041512, 0.0001466 ],
       [0.00039121, 0.00041872, 0.00014826],
       [0.0003934 , 0.00041616, 0.00014925],
       [0.00039074, 0.00041685, 0.00014472]]), array([-0.89454277, -0.89454277, -0.89454277, -0.89454277]))
           fun: -0.894542772861357
       message: 'Optimization terminated successfully.'
          nfev: 47
           nit: 19
        status: 0
       success: True
             x: array([0.00038735, 0.00041512, 0.0001466 ])

In [13]:
d1 = load('knc.npy') + load('rfc.npy') + load('svcc.npy')
d2 = load('svcc.npy') + load('lrc.npy') + load('rfc.npy')
d3 = load('svcc.npy') + load('knc.npy') + load('rfc.npy')
d4 = load('svcc.npy') + load('knc.npy') + load('lrc.npy')

In [14]:
res_cl_report(1, load('rfc.npy'), 1, load('rfc.npy'), 1, load('svcc.npy'))

             precision    recall  f1-score   support

          0       0.79      0.82      0.81       168
          1       0.94      0.95      0.94       167
          2       0.90      0.85      0.88       169
          3       0.70      0.68      0.69       171
          4       0.88      0.87      0.87       171
          5       0.91      0.89      0.90       169
          6       0.79      0.81      0.80       172
          7       0.75      0.76      0.76       169

avg / total       0.83      0.83      0.83      1356



In [15]:
save('best_coef2', [1.30831118e-04, 9.98999037e-06, 4.07361090e-04, 3.26582233e-05,
       8.15057852e-05])

In [16]:
data = np.hstack((d1, d2, d3, d4))

In [17]:
svcc = SVC(C=1, gamma=1, kernel = 'rbf', probability=True)
lrc = LogisticRegression(C = 430)
knc = KNeighborsClassifier(weights = 'distance', n_neighbors=27, metric = 'manhattan')

In [18]:
rfc.fit(data, load('labels.npy'))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=23, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1300, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [19]:
data = data[:, rfc.feature_importances_ > np.median(rfc.feature_importances_)*1]
shape(data)

(1356, 16)

In [20]:
lrc_ridge_scoring1 = cross_validation.cross_val_score(rfc, data, load('labels.npy'),
                                                  scoring='f1_weighted', cv = 5)
print('lrc --  mean:{}, max:{}, min:{}, std:{}'.format(lrc_ridge_scoring1.mean(), lrc_ridge_scoring1.max(),
                                                     lrc_ridge_scoring1.min(), lrc_ridge_scoring1.std()))

lrc --  mean:0.8486418465857042, max:0.8758065749865688, min:0.80669180558144, std:0.02666024137961361
